In [1]:
# @title Import necessary modules
import influxdb_client
print("Influxdb_client Version : {}".format(influxdb_client.__version__))
import pandas as pd
from datetime import datetime
import numpy as np
%matplotlib inline
from ipywidgets import interactive
import ipywidgets as widgets
import matplotlib.pyplot as plt

import time
from influxdb_client import InfluxDBClient
from IPython.display import display
import bokeh
print("Bokeh Version : {}".format(bokeh.__version__))
from bokeh.io import (output_notebook,
                      show,
                      push_notebook,
                      curdoc,
                      )
from bokeh.models import (ColumnDataSource,
                          DatetimeTickFormatter,
                          NumeralTickFormatter)
from bokeh.plotting import figure

# Bokeh Time formatter
formatter = DatetimeTickFormatter(

    seconds = "%H:%M:%S",
    minsec = "%H:%M:%S",
    minutes = "%H:%M:%S",
    hourmin = "%H:%M:%S",
    hours= "%H:%M:%S",
    days= "%H:%M:%S",
    months="%H:%M:%S",

)

Influxdb_client Version : 1.48.0
Bokeh Version : 3.6.1


In [16]:
# @title
client = InfluxDBClient(url='http://epics.ipfn.tecnico.ulisboa.pt:8086',
                        token='kJXa2Ok88yE8nSEyh2lStvs6v3vWYGhb90XL3M-gGhuysJWj36XaJkQaXSnGUa7h6FdkiIhZJ3JAYVCpc0390w==',
                        org='67c974ed52348dd6')
query_api = client.query_api()
wExp = widgets.Dropdown(
    options=['calorimetry', 'calorimetryEsp32', ],
    value='calorimetry',
    description='Experiment:',
    disabled=False,
)
display(wExp)
wAcq = widgets.BoundedIntText(
    value=120,
    min=1,
    max=360,
    step=1,
    description='Acq Time/s:',
    disabled=False
)
display(wAcq)

Dropdown(description='Experiment:', options=('calorimetry', 'calorimetryEsp32'), value='calorimetry')

BoundedIntText(value=120, description='Acq Time/s:', max=360, min=1)

In [18]:
# @title Live Temperature Data
output_notebook()

LIVE_WINDOW = 5 # in min
query_temp = (f'from(bucket:"ardu-rasp") |> range(start: -{LIVE_WINDOW:d}m) '
    '|> filter(fn: (r) => r._measurement == "ds18b20Sensors") '
    f'|> filter(fn: (r) => r.experiment == "{wExp.value}") '
    '|> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")')
temperature_table_df = query_api.query_data_frame(org='67c974ed52348dd6', query=query_temp)
source = ColumnDataSource(temperature_table_df)
curdoc().theme = "dark_minimal"
fig = figure(title=f"ARDUINO-RASP Live Temperature Plot {EXPERIMENT:s}",
            x_axis_type="datetime",
            sizing_mode="stretch_width",
            x_axis_label='Time',
            y_axis_label='ºC',
            y_range=(10, 100),)

fig.line(x='_time', y='tempBlue', source=source, color="blue", line_width=2)
fig.line(x='_time', y='tempRed', source=source, color="red", line_width=2)
fig.xaxis[0].formatter = formatter  # DatetimeTickFormatter() # months="%b %Y")
t=show(fig, notebook_handle=True)

def update_plot():
    temperature_table_df = query_api.query_data_frame(org='67c974ed52348dd6', query=query_temp)
    source.data = temperature_table_df
    # source.stream(new_tables_df)
    push_notebook(handle=t)

for i in range(wAcq.value):
    update_plot()
    time.sleep(1)

Loading BokehJS ...

In [ ]:
# @title Save CSV File (You may need to download file from 'Files' section)
T_mat = np.array([tRed, tBlue])
#filename = f"Temperature_log_{datetime.now():%Y_%m_%d_%H-%M-%S}.csv"
filename = f"Temperature_log_{START:s}.csv"
np.savetxt(filename, T_mat.T, delimiter=",")